In [17]:
from model_analysis import LoadData,Tokenization,ErrorAnalysis,ModelComparation
from transformers import BertTokenizer, BertModel,BertForSequenceClassification
from utils import Config
from utils import *


In [18]:
configuration = Config()
tokenizer = BertTokenizer.from_pretrained(configuration.pretrained_model, 
                                              do_lower_case=configuration.do_lower_case)

modelBert = BertForSequenceClassification.from_pretrained(configuration.pretrained_model,
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
modelBert.to(configuration.device)
modelBert.load_state_dict(torch.load(f'C:/Users/Miguel Alejandro/Documents/3er Año/Segundo Período/SRI/Proyecto/_BERT_epoch_6.model', map_location=torch.device('cpu')))

modelTrained = BertForSequenceClassification.from_pretrained(configuration.pretrained_model,
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
modelTrained.to(configuration.device)

modelBert.load_state_dict(torch.load(f'C:/Users/Miguel Alejandro/Documents/3er Año/Segundo Período/SRI/Proyecto/_BERT_epoch_6.model', map_location=torch.device('cpu')))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


FileNotFoundError: [Errno 2] No such file or directory: './_BERT_epoch_6.model'

In [13]:
from sklearn.model_selection import train_test_split
df = LoadData("C:/Users/Miguel Alejandro/Documents/3er Año/Segundo Período/SRI/Proyecto/datafiniti-hotel-reviews/7282_1.csv")


df = df[["address", "categories","city","country","name","province","reviews.rating","reviews.text"]]


for x in df.index:
    if(isinstance(df["reviews.text"][x],float)):
        df.drop(x,inplace=True)
df = df.dropna()
df.head()

# encode label and mapping label name
df["label"] = df["reviews.rating"].apply(lambda x: label_encode(x))#.astype(int)
df["label_name"] = df["label"].apply(lambda x: label2name(x))
clean = CleanText()
# clean text, lowercase and remove punk
df["reviews.text"] = df["reviews.text"].apply(lambda x: remove_punct(clean(remove_emoji(x).lower())[0][0]))
                                            
df = df.dropna()
train_df_, y = train_test_split(df,
                                    test_size=0.10,
                                    random_state=configuration.random_state,
                                    stratify=df.label.values)

c:\Users\Miguel Alejandro\Documents\3er Año\Segundo Período\SRI\Proyecto\Hotel-Recommendation-System\model_analysis.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)# Drop any rows with missing values (NaN) from the 'data' DataFrame


In [14]:
#%store -r y
print(y)
encoded_data = Tokenization(y,configuration,tokenizer)
print(encoded_data)


                       address  \
19468           265 N Millward   
27281           15 Howard Blvd   
16084         2900 S Casino Dr   
14094     11400 Common Oaks Dr   
34520             425 Ocean Dr   
...                        ...   
15985  2120 Claude Bailey Pkwy   
3160      747 Buffalo Creek Rd   
22273            1282 W 7th St   
31294         1606 Central Ave   
15332             95 Route 17k   

                                              categories             city  \
19468                                             Hotels          Jackson   
27281                                             Hotels  Mount Arlington   
16084                                             Hotels         Laughlin   
14094                                             Hotels          Raleigh   
34520                                             Hotels      Miami Beach   
...                                                  ...              ...   
15985                 Travel & Transport,Hotels & Mo

C:\Users\Miguel Alejandro\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': tensor([[ 101, 1045, 2031,  ...,    0,    0,    0],
        [ 101, 1045, 2074,  ...,    0,    0,    0],
        [ 101, 4638, 1999,  ...,    0,    0,    0],
        ...,
        [ 101, 2057, 7039,  ...,    0,    0,    0],
        [ 101, 2190, 2477,  ...,    0,    0,    0],
        [ 101, 1062, 5714,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [16]:
mertics_1 = ErrorAnalysis(y,encoded_data,configuration,modelBert)

AttributeError: 

In [8]:
mertics_2 = ErrorAnalysis(y,encoded_data,configuration,modelTrained)

AttributeError: 

In [ ]:
ModelComparation(metrics_1,metrics_2)

In [ ]:
df = LoadData("C:/Users/Miguel Alejandro/Documents/3er Año/Segundo Período/SRI/Proyecto/datafiniti-hotel-reviews/7282_1.csv")
df.head()



c:\Users\Miguel Alejandro\Documents\3er Año\Segundo Período\SRI\Proyecto\Hotel-Recommendation-System\ranking.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)# Drop any rows with missing values (NaN) from the 'data' DataFrame
C:\Users\Miguel Alejandro\AppData\Roaming\Python\Python311\site-packages\transformers\tokenization_utils_base.py:2606: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.war

RuntimeError: [enforce fail at alloc_cpu.cpp:80] data. DefaultCPUAllocator: not enough memory: you tried to allocate 55091134464 bytes.

In [ ]:
# entrenamiento

In [ ]:
%store -r y
# data = Processing(BertNewTokenizer(df,configuration,tokenizer),configuration,)
BertProcData = Processing(BertNewTokenizer(df,configuration,tokenizer),configuration,))

no stored variable or alias y


TypeError: Processing() missing 3 required positional arguments: 'encoded_data', 'config', and 'model'